In [158]:
URL = "https://www.youtube.com/watch?v=y316Fh2Qlzk"

In [159]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import random

In [160]:
# 웹드라이버 설정
browser = webdriver.Chrome()
browser.get(URL)  # 유튜브 비디오 URL 입력

time.sleep(2)  # 페이지 로딩 대기

# 댓글 섹션까지 스크롤
browser.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
time.sleep(2)  # 동적 로딩 대기

In [161]:
# 스크롤 다운하여 모든 댓글 로드
last_height = browser.execute_script("return document.documentElement.scrollHeight")
while True:
    browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(random.uniform(1.5, 3.0))  # 로딩과 다양성을 위해 랜덤 대기 시간 적용
    new_height = browser.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [162]:
# 댓글 요소 수집
comments = browser.find_elements(By.XPATH, '//*[@id="content-text"]')
usernames = browser.find_elements(By.XPATH, '//*[@id="author-text"]')
likes = browser.find_elements(By.XPATH, '//*[contains(@id, "vote-count-middle")]')
#dates = browser.find_elements(By.XPATH, '//*[@id="header-author"]//yt-formatted-string/a')
#reply_counts = browser.find_elements(By.XPATH, '//ytd-comment-thread-renderer//span[@id="vote-count-middle"]/following-sibling::span')

# DataFrame 생성
data = {
    # '작성일': [date.text for date in dates],
    '사용자이름': [username.text for username in usernames],
    'comment': [comment.text for comment in comments],
    '좋아요수': [like.text if like.text != '' else '0' for like in likes],
    # '답글수': [reply.text.split()[0] if reply.text != '' else '0' for reply in reply_counts]
}
df = pd.DataFrame(data)

# DataFrame 확인
print(df)


                사용자이름                                            comment 좋아요수
0    @user-xf5os8nn3i  신이시여\n우리 유작가님께 200살의 수명을 주십시오\n우리곁에 오래오래 함께 할수...  132
1        @rosemom8309  두둥~~~~\n드디어, 마침내 ~~!!\n유작가님의 흐믓한  미소!!  ㅎㅎ\n조변...   93
2    @user-kr6nx5no2s  유시민 작가님이 그 자리에 그저 계셔주신 것만으로도 든든합니다. 감사합니다.\n항상...  140
3     @danielle-wo5rx             언제들어도 재미있는 지리이야기!\n유작가님과 함께라서 더욱 즐겁습니다  114
4    @user-pd1xr2op1n  위대한 정치가인 노무현 대통령님 존경하고 넘 그립습니다.\n박학다식 유작가님, 박교...  172
5     @rayawesome6762                잘 들었습니다. 조수진 변호사님 나오니까 훨씬 즐겁고 재밌다요~  119
6     @nayoungkim5688  유시민작가님이 계셔서 든든합니다.고맙습니다.감사합니다\n조수진변호사님 항상 감사합니...  147
7    @user-us9ol9dc3j     제가 젤 좋아한 책중 하나입니다.\n잘 듣겠습니다 \n감사해요 알릴레오 북스 ♡.♡   71
8    @user-ve7yc1rn5q                         덕분에 매번 교양을 쌓을 수 있습니다 감사합니다   60
9         @hunjukim20                             역시 아는것이 힘입니다 유시민작가님은 짱   29
10   @user-jf4ik8jy1s  진행자와 손님 위치를 바꾸셔서 너무 잘 되었습니다!!\n보는 시선이 훨씬 편해졌네요...   11
11            @sj3865                        총균쇠. 이번 기회에 읽어야겠다. 

In [163]:
# pip install wordcloud

In [164]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
%matplotlib inline

# plt 한국어폰트설정맥북환경 애플고딕
rc('font', family='AppleGothic')


In [165]:
df['comment'] = df['comment'].str.replace('[^가-힣]', ' ', regex = True)
df['comment']

0      신이시여 우리 유작가님께    살의 수명을 주십시오 우리곁에 오래오래 함께 할수 있도록
1     두둥     드디어  마침내      유작가님의 흐믓한  미소       조변님도 ...
2     유시민 작가님이 그 자리에 그저 계셔주신 것만으로도 든든합니다  감사합니다  항상 ...
3                 언제들어도 재미있는 지리이야기  유작가님과 함께라서 더욱 즐겁습니다
4     위대한 정치가인 노무현 대통령님 존경하고 넘 그립습니다  박학다식 유작가님  박교수...
5                   잘 들었습니다  조수진 변호사님 나오니까 훨씬 즐겁고 재밌다요 
6     유시민작가님이 계셔서 든든합니다 고맙습니다 감사합니다 조수진변호사님 항상 감사합니다...
7          제가 젤 좋아한 책중 하나입니다  잘 듣겠습니다  감사해요 알릴레오 북스    
8                            덕분에 매번 교양을 쌓을 수 있습니다 감사합니다
9                                역시 아는것이 힘입니다 유시민작가님은 짱
10    진행자와 손님 위치를 바꾸셔서 너무 잘 되었습니다   보는 시선이 훨씬 편해졌네요    
11                           총균쇠  이번 기회에 읽어야겠다   감사합니다 
12                                     세분 말씀 감사히 잘들었습니다
13           자리배치가 좋아졌네요   오늘도 재미나게 잘 들었습니다  다음주도 기대기대 
14    이번편 총 균 세 편을 보다가 문득 두달전쯤에 유튜브에서 본게 생각났는데      ...
15    유시민작가님  유튜브 어디든 어준총수 겸공이든 매불쇼든 어디든 좀 나와서 현상황에 ...
16      총균쇠    도서관에서 늘 못본척했는데   오늘 방송 듣고 보니 꼭 읽어야겠네요   
17                유익한 방송 너무 좋아요 다음회를 기다립니다 유시민작가

In [166]:
# 댓글 컬럼을 하나의 문자열로 결합
all_comments_string = df['comment'].str.cat(sep=' ')

# 결과 출력
all_comments_string

'신이시여 우리 유작가님께    살의 수명을 주십시오 우리곁에 오래오래 함께 할수 있도록 두둥     드디어  마침내      유작가님의 흐믓한  미소       조변님도  엄청 기대에 찬 목소리     시청하는 나는  아는 만큼 보이고 보이는 만큼 들린다  를  깨닫고 있음  역시 알릴레오의 책 선정은 최고    정말정말 고맙습니다  알릴레오 알라뷰   유시민 작가님이 그 자리에 그저 계셔주신 것만으로도 든든합니다  감사합니다  항상 건강하셔요      언제들어도 재미있는 지리이야기  유작가님과 함께라서 더욱 즐겁습니다 위대한 정치가인 노무현 대통령님 존경하고 넘 그립습니다  박학다식 유작가님  박교수님  조변호사님 알찬 정보 고맙습니다   잘 들었습니다  조수진 변호사님 나오니까 훨씬 즐겁고 재밌다요  유시민작가님이 계셔서 든든합니다 고맙습니다 감사합니다 조수진변호사님 항상 감사합니다 두 분 언제나 존경합니다 제가 젤 좋아한 책중 하나입니다  잘 듣겠습니다  감사해요 알릴레오 북스     덕분에 매번 교양을 쌓을 수 있습니다 감사합니다 역시 아는것이 힘입니다 유시민작가님은 짱 진행자와 손님 위치를 바꾸셔서 너무 잘 되었습니다   보는 시선이 훨씬 편해졌네요     총균쇠  이번 기회에 읽어야겠다   감사합니다  세분 말씀 감사히 잘들었습니다 자리배치가 좋아졌네요   오늘도 재미나게 잘 들었습니다  다음주도 기대기대  이번편 총 균 세 편을 보다가 문득 두달전쯤에 유튜브에서 본게 생각났는데      에서         일 방영한 사람과 사람들 프로그램에  백 년을 살아보니  편에 시골에 사시는    세 최돈춘 어르신 삶에 대해서 나오는데   가  연세가 올해 어떻게 되세요  라고 물으니 어르신이    살이야     살 내가 대정 원년에 태어났어  강릉에서 나보다 나이 많은 사람이 없어   그러면서 주민등록증을 보여주며 예전에 없었단 말이여  해방이후 자유당이 지나가고 한참지나서 그때 한거지  고종 황제가 내가  살에 사망했다고 노인들이 우리 아버지보다 많은 

In [167]:
# !pip install konlpy

In [168]:
# 자바 버전 이슈로 실행이 안됨


# from konlpy.tag import Okt
# import pandas as pd

# # Okt 형태소 분석기 인스턴스 생성
# okt = Okt()


# # 모든 댓글을 하나의 문자열로 결합
# comments_string = df['comment'].str.cat(sep=' ')

# # 명사 추출
# nouns = okt.nouns(comments_string)

# # 추출된 명사를 다시 하나의 문자열로 결합
# nouns_string = ' '.join(nouns)


# # 워드 클라우드 생성
# wordcloud = WordCloud(font_path = '/System/Library/Fonts/AppleGothic.ttf',
#                       width = 800,
#                       height = 400, 
#                       background_color ='white').generate(nouns_string)

# # 워드 클라우드 표시
# plt.figure(figsize=(100, 50))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.show()

In [169]:
# pip install soynlp

In [172]:
from soynlp.noun import LRNounExtractor_v2
import pandas as pd

# 데이터프레임 생성
data = {
    'comment': [
        "이 비디오 너무 유익해요, 감사합니다!",
        "정말 도움이 되는 튜투리얼이었어요, 많이 배웠습니다!",
        "이 주제에 대해 더 많은 비디오를 만들어주실 수 있나요?",
        "설명이 명확하지 않아요, 하지만 감사합니다.",
        "훌륭한 설명, 명확하고 간결해요!",
        "이 내용은 제가 주제를 이해하는데 도움이 되지 않았어요.",
        "감사해요, 이 비디오는 정말 눈을 뜨게 하는 내용이었어요!",
        "다음 번에는 예시를 더 많이 제공해 주세요."
    ]
}
df = pd.DataFrame(data)

# 모든 댓글을 하나의 문자열로 결합
comments_string = df['comment'].str.cat(sep=' ')

# soynlp 명사 추출기 인스턴스 생성
noun_extractor = LRNounExtractor_v2()
nouns = noun_extractor.train_extract([comments_string])  # 배열 형태로 입력

# 추출된 명사를 다시 하나의 문자열로 결합
nouns_string = ' '.join(nouns.keys())  # 추출된 명사들의 key만 사용

# 결과 출력
print(nouns_string)


[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 45 from 1 sents. mem=0.068 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=52, mem=0.069 Gb
[Noun Extractor] batch prediction was completed for 16 words


[Noun Extractor] checked compounds. discovered 0 compounds
[Noun Extractor] postprocessing detaching_features : 4 -> 4
[Noun Extractor] postprocessing ignore_features : 4 -> 4
[Noun Extractor] postprocessing ignore_NJ : 4 -> 4
[Noun Extractor] 4 nouns (0 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=0.069 Gb                    
[Noun Extractor] 15.38 % eojeols are covered
비디오 주제 설명 제
